# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [6]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [7]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [8]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [11]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'A RBTKANL )SNAEMLGGAQL/DT(CSOE SE', 'calculationPrice': 'close', 'open': 128.53, 'openTime': 1642733471767, 'openSource': 'cofiflia', 'close': 126.37, 'closeTime': 1638833115297, 'closeSource': 'fiiflcao', 'high': 128.53, 'highTime': 1634605365829, 'highSource': 'dr dp iai euce15mylntee', 'low': 125.39, 'lowTime': 1694511058651, 'lowSource': 'eeiudi15pctldney ra  em', 'latestPrice': 125.79, 'latestSource': 'Close', 'latestTime': 'May 28, 2021', 'latestUpdate': 1656159105856, 'latestVolume': 73564041, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 128.14, 'delayedPriceTime': 1624140017051, 'oddLotDelayedPrice': 128.57, 'oddLotDelayedPriceTime': 1675739856146, 'extendedPrice': 130.08, 'extendedChange': 0.01, 'extendedChangePercent': 8e-05, 'extendedPriceTime': 1655337015662, 'previousClose': 128.69, 'previousVolume': 96768274, 'change': -0.7, 'changePercent': -0.00556, 'volume':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [89]:
price = data['latestPrice']
market_cap =data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [90]:
my_columns = ['Ticker', 'Stock Price','Market Cap','No of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,No of shares to buy


In [91]:
final_dataframe.append(
      pd.Series(
      [
          symbol,
          price,
          market_cap/1000000000,
          'N/A'
          
      ],
          index= my_columns
      ),
    
      ignore_index=True
)

,Ticker,Stock Price,Market Cap,No of shares to buy
0,AAPL,129.79,2165.000756,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [53]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']: 
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
        index = my_columns
    ),
        ignore_index =True
    )

In [54]:
final_dataframe

,Ticker,Stock Price,Market Cap(Billion $),No of shares to buy
0,A,144.330,42512735237,N/A
1,AAL,25.016,16325635247,N/A
2,AAP,196.600,12815350541,N/A
3,AAPL,127.800,2097461786735,N/A
4,ABBV,115.990,201827027187,N/A
...,...,...,...,...
500,YUM,122.000,36378862995,N/A
501,ZBH,171.900,36391184052,N/A
502,ZBRA,514.160,27638632207,N/A
503,ZION,58.940,9631700622,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [92]:
def chunks(lst,n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [93]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['latestPrice'],
            'N/A'
            
        ],
            index =my_columns
        ),ignore_index = True
        )
final_dataframe

,Ticker,Stock Price,Market Cap,No of shares to buy
0,A,142.87,142.87,N/A
1,AAL,25.26,25.26,N/A
2,AAP,194.97,194.97,N/A
3,AAPL,130.62,130.62,N/A
4,ABBV,114.60,114.60,N/A
...,...,...,...,...
500,YUM,125.08,125.08,N/A
501,ZBH,172.25,172.25,N/A
502,ZBRA,519.64,519.64,N/A
503,ZION,60.73,60.73,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [94]:
 portfolio_size = input("Enter the value of your portfoilio:")
    
try:
    val =float(portfolio_size)
    print(val)
except ValueError:
    print('That is not a number! \n Please try again:')
    portfolio_size = input("Enter the value of your portfoilio:")
    val = float(portfolio_size)

Enter the value of your portfoilio:1000000
1000000.0


In [95]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'No of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Cap,No of shares to buy
0,A,142.87,142.87,13
1,AAL,25.26,25.26,78
2,AAP,194.97,194.97,10
3,AAPL,130.62,130.62,15
4,ABBV,114.60,114.60,17
...,...,...,...,...
500,YUM,125.08,125.08,15
501,ZBH,172.25,172.25,11
502,ZBRA,519.64,519.64,3
503,ZION,60.73,60.73,32


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [120]:
writer = pd.ExcelWriter('Recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [121]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [122]:
#writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
#writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [125]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Cap', dollar_format],
    'D':['No of shares to buy', integer_format]    
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [126]:
writer.save()  